In [1]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

### 1. Load file

In [1]:
import os
import pandas as pd

pd.options.display.max_colwidth = 100000000
# C:/Users/doudi/OneDrive/Documents/TMU-GIDS/機器學習與深度學習/Final term
os.chdir('C:/Users/doudi/OneDrive/Documents/2019_MLDL_final/reviews_data/')
path = 'reviews_191226_52555_final.csv'
df = pd.read_csv(path)
df.head()

,game_id,useful_num,funny_num,user_name,games_owned,reviews_written,recommended,hours_played,review_date,text
0,280720,8,0,76561198206307425,128,51,1,40.3,"15 August, 2019",雖說是搶先體驗，但是玩了這麼久，沒感覺有遇到bug...算是很有誠意了。 漢化上面大部分都好了，漏了一個星球的主線沒有而已，這個是即時制、可調時間速度與暫停的遊戲，也可以很快地讀取、存檔、關閉，因為外星人侵略性不是很大，也沒有軍事部隊，主要是透過製作的道具或是購買股權的方式去吞併削弱對手。 當然也可以選擇故意破壞環境，讓大災變替你把所有對手趕跑之後自己再慢慢重新收拾XDD 可能就背景音樂的部分比較不足而已，其他畫面啦、遊戲內容之類的，因為沒在打廣告，所以只要三百多而已，很值啦。
1,410570,0,0,76561198424252800,38,2,1,0.7,"7 October, 2019",good
2,410570,0,0,76561198300261802,16,3,1,0.7,"23 June, 2017",遊戲是挺耐玩的，不過玩得有點累，美術方面，可能因為從手機版移植來，所以遠景有點假。但整體而言，印象還不錯，科幻迷可以考慮。
3,410570,0,0,76561198307052400,80,6,1,6.6,"30 September, 2016",這是我玩過幾款vr遊戲中我個人覺得很不錯的一款遊戲
4,410570,0,0,76561198307075359,102,4,1,9.3,"21 September, 2016",优点：整体做的非常好，沉浸感比较强，价格合理。不晕。缺点：整体游戏时间有点短，并且不能调整难度。期望：steam赶紧销售EVE:Valkyrie。 Gunjack增加DLC。


### 2. Preprocessing

In [4]:
# 先安裝 PYTORCH
# https://pytorch.org/
# 如果有獨顯可以安裝GPU版本，如何安裝 pytorch GPU 說明文 : https://zhuanlan.zhihu.com/p/54350088
# monpa 說明文 : https://github.com/monpa-team/monpa
# !conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
# !pip install monpa
import re
import monpa
import jieba
from tqdm import tqdm_notebook as tqdm

# stop word
stw=[]
with open('C:/Users/doudi/Downloads/chineses_stop_word.txt', 'r') as file:
    for data in file.readlines():
        data = data.strip()
        stw.append(data)
# remove stopword
def movestopwords(s):
    outstr = []
    if s is not None:
        for word in s:
            if word not in stw:
                if word != '\t'and'\n':
                    outstr.append(word)
                    # outstr += " "
    return outstr


#  分割句子，因為猛趴只吃最常140字元的短文
def LongCut(long_sentence, split_char):
    seg = []
    for item in long_sentence.split(split_char):
        if item != "\n": seg.extend(monpa.cut(str(item+split_char)))
    return seg[:-1]

# 切詞(這會花很久的時間，建議先抽樣測試)
def monpa_seg(corpus, stop):
    result_co = []
    for item in tqdm(corpus.iloc[:,9]):
        sentence = LongCut(str(item), '，')
        if sentence is not None:
            sentence = [x.strip(' ') for x in sentence]
        result_co.append(movestopwords(sentence))
    corpus['text_seg_monpa_no_stw']=result_co
    return corpus

def jieba_seg(corpus, stop):
    c = []
    for s in tqdm(corpus.iloc[:,9]):
        seg = jieba.cut(s, cut_all=False)
        if seg is not None:
            seg = [x.strip(' ') for x in seg]
        c.append(movestopwords(seg))
    corpus['text_seg_jieba_no_stw'] = c
    
    return corpus




In [5]:
# sam = df.iloc[0:1000,:]

In [8]:
Corpus = monpa_seg(df , stw)
Corpus = jieba_seg(df , stw)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


AttributeError: 'float' object has no attribute 'decode'

In [9]:
Corpus.head()

,game_id,useful_num,funny_num,user_name,games_owned,reviews_written,recommended,hours_played,review_date,text,text_seg_monpa_no_stw
0,280720,8,0,76561198206307425,128,51,1,40.3,"15 August, 2019",雖說是搶先體驗，但是玩了這麼久，沒感覺有遇到bug...算是很有誠意了。 漢化上面大部分都好了，漏了一個星球的主線沒有而已，這個是即時制、可調時間速度與暫停的遊戲，也可以很快地讀取、存檔、關閉，因為外星人侵略性不是很大，也沒有軍事部隊，主要是透過製作的道具或是購買股權的方式去吞併削弱對手。 當然也可以選擇故意破壞環境，讓大災變替你把所有對手趕跑之後自己再慢慢重新收拾XDD 可能就背景音樂的部分比較不足而已，其他畫面啦、遊戲內容之類的，因為沒在打廣告，所以只要三百多而已，很值啦。,"[雖說, 搶先, 體驗, 但是, 玩, 這麼, 久, 感覺, 遇到, bug, ..., 算是, 誠意, 漢化, 上面, 大部分, 好了, 漏, 星球, 主線, 而已, 即時制, 調, 時間, 速度, 暫停, 遊戲, 快, 讀取, 存檔, 關閉, 外星人, 侵略性, 軍事, 部隊, 主要, 透過, 製作, 道具, 或是, 購買, 股權, 方式, 吞併, 削弱, 對手, 當然, 選擇, 故意, 破壞, 環境, 災變, 替, 所有, 對手, 趕跑, 之後, 慢慢, 重新, 收拾, XDD, 背景, 音樂, 部分, 比較, 不足, 而已, 其他, 畫面, 啦, 遊戲, 內容, 之類, 打廣告, 只要, 三百多, 而已, 值, 啦]"
1,410570,0,0,76561198424252800,38,2,1,0.7,"7 October, 2019",good,[good]
2,410570,0,0,76561198300261802,16,3,1,0.7,"23 June, 2017",遊戲是挺耐玩的，不過玩得有點累，美術方面，可能因為從手機版移植來，所以遠景有點假。但整體而言，印象還不錯，科幻迷可以考慮。,"[遊戲, 挺, 耐玩, 不過, 玩, 有點, 累, 美術, 方面, 手機版, 移植, 遠景, 有點, 假, 整體, 而言, 印象, 不錯, 科幻迷, 考慮]"
3,410570,0,0,76561198307052400,80,6,1,6.6,"30 September, 2016",這是我玩過幾款vr遊戲中我個人覺得很不錯的一款遊戲,"[玩, 過, 幾, 款, vr, 遊戲, 個人, 覺得, 不錯, 款, 遊戲]"
4,410570,0,0,76561198307075359,102,4,1,9.3,"21 September, 2016",优点：整体做的非常好，沉浸感比较强，价格合理。不晕。缺点：整体游戏时间有点短，并且不能调整难度。期望：steam赶紧销售EVE:Valkyrie。 Gunjack增加DLC。,"[优点, 整, 体, 非常, 沉浸, 感, 较强, 价格, 合理, 晕, 缺点, 整, 体, 游戏, 时, 间, 点, 短, 并且, 不能, 调整, 难度, 期望, steam, 赶紧销, 售, EVE, Valkyrie, Gunjack, 增加, DLC]"


> sklearn TFIDF不用斷詞，不過為了深度學習的資料，還是要先斷乾淨XD

### 3. Baseline model

In [198]:
X = list(df['text'])
Y = list(df['recommended'])


# Cross_cal
def baseline_model(train, test):
    from sklearn.naive_bayes import BernoulliNB, MultinomialNB
    from sklearn import svm
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from xgboost import XGBClassifier
    
    from sklearn.model_selection import cross_val_score
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    tfidf_vectorizer = TfidfVectorizer()
    
    Models = ['Bernoulli NB','Multinomial NB','Svm (linear)','Logistic Regression',
              'Random Forest','kNN','Decision Tree','XG Boost']
    function = [BernoulliNB(),MultinomialNB(),svm.SVC(kernel="linear"),LogisticRegression(),
              RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier(),
                XGBClassifier()]
    perform = []
    
    for i in range(len(function)):
        model = function[i]
        performance = cross_val_score(model, tfidf_vectorizer.fit_transform(train), test, cv=10,
                                      scoring='accuracy')
        func = str(function[i])
        print("==== ", func[0:func.index('(')], " ====")
        print("avg-accuracy: {0}".format(performance.mean()))
        print('\n')
        perform.append(performance.mean())

    result_table = pd.DataFrame({"Models":Models,"Result":perform})
    return result_table

result = baseline_model(X, Y)
result

====  BernoulliNB  ====
avg-accuracy: 0.7905440450015293


====  MultinomialNB  ====
avg-accuracy: 0.8210084803471778


====  SVC  ====
avg-accuracy: 0.8260900521112863


====  LogisticRegression  ====
avg-accuracy: 0.8227066056681844




C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\doudi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default

====  RandomForestClassifier  ====
avg-accuracy: 0.825544304692324


====  KNeighborsClassifier  ====
avg-accuracy: 0.8193103189580052


====  DecisionTreeClassifier  ====
avg-accuracy: 0.8244142890531674


====  XGBClassifier  ====
avg-accuracy: 0.8226938375374389




,Models,Result
0,Bernoulli NB,0.790544
1,Multinomial NB,0.821008
2,Svm (linear),0.826090
3,Logistic Regression,0.822707
4,Random Forest,0.825544
5,kNN,0.819310
6,Decision Tree,0.824414
7,XG Boost,0.822694


In [60]:
os.chdir('C:/Users/doudi/Downloads/AFF540DC.Unpacker_v7353qx4kg3sa!App/filename.7z/')
data = pd.read_csv('filename.csv')

In [58]:
data.head()

,game_id,useful_num,funny_num,user_name,games_owned,reviews_written,recommended,hours_played,review_date,text,text_seg_monpa_no_stw
0,280720.0,8,0,76561198206307425,128,51,1,40.3,"15 August, 2019",雖說是搶先體驗，但是玩了這麼久，沒感覺有遇到bug...算是很有誠意了。 漢化上面大部分都好了，漏了一個星球的主線沒有而已，這個是即時制、可調時間速度與暫停的遊戲，也可以很快地讀取、存檔、關閉，因為外星人侵略性不是很大，也沒有軍事部隊，主要是透過製作的道具或是購買股權的方式去吞併削弱對手。 當然也可以選擇故意破壞環境，讓大災變替你把所有對手趕跑之後自己再慢慢重新收拾XDD 可能就背景音樂的部分比較不足而已，其他畫面啦、遊戲內容之類的，因為沒在打廣告，所以只要三百多而已，很值啦。,"['雖說', '是', '搶先', '體驗', '，', '但是', '玩', '了', '這麼', '久', '，', '沒', '感覺', '有', '遇到', 'bug', '...', '算是', '很', '有', '誠意', '了', '。', '漢化', '上面', '大部分', '都', '好了', '，', '漏', '了', '一', '個', '星球', '的', '主線', '沒有', '而已', '，', '這', '個', '是', '即時制', '、', '可', '調', '時間', '速度', '與', '暫停', '的', '遊戲', '，', '也', '可以', '很', '快', '地', '讀取', '、', '存檔', '、', '關閉', '，', '因為', '外星人', '侵略性', '不', '是', '很', '大', '，', '也', '沒有', '軍事', '部隊', '，', '主要', '是', '透過', '製作', '的', '道具', '或是', '購買', '股權', '的', '方式', '去', '吞併', '削弱', '對手', '。', '當然', '也', '可以', '選擇', '故意', '破壞', '環境', '，', '讓', '大', '災變', '替', '你', '把', '所有', '對手', '趕跑', '之後', '自己', '再', '慢慢', '重新', '收拾', 'XDD', '可能', '就', '背景', '音樂', '的', '部分', '比較', '不足', '而已', '，', '其他', '畫面', '啦', '、', '遊戲', '內容', '之類', '的', '，', '因為', '沒', '在', '打廣告', '，', '所以', '只要', '三百多', '而已', '，', '很', '值', '啦', '。']"
1,410570.0,0,0,76561198424252800,38,2,1,0.7,"7 October, 2019",good,['good']
2,410570.0,0,0,76561198300261802,16,3,1,0.7,"23 June, 2017",遊戲是挺耐玩的，不過玩得有點累，美術方面，可能因為從手機版移植來，所以遠景有點假。但整體而言，印象還不錯，科幻迷可以考慮。,"['遊戲', '是', '挺', '耐玩', '的', '，', '不過', '玩', '得', '有點', '累', '，', '美術', '方面', '，', '可能', '因為', '從', '手機版', '移植', '來', '，', '所以', '遠景', '有點', '假', '。', '但', '整體', '而言', '，', '印象', '還', '不錯', '，', '科幻迷', '可以', '考慮', '。']"
3,410570.0,0,0,76561198307052400,80,6,1,6.6,"30 September, 2016",這是我玩過幾款vr遊戲中我個人覺得很不錯的一款遊戲,"['這', '是', '我', '玩', '過', '幾', '款', 'vr', '遊戲', '中', '我', '個人', '覺得', '很', '不錯', '的', '一', '款', '遊戲']"
4,410570.0,0,0,76561198307075359,102,4,1,9.3,"21 September, 2016",优点：整体做的非常好，沉浸感比较强，价格合理。不晕。缺点：整体游戏时间有点短，并且不能调整难度。期望：steam赶紧销售EVE:Valkyrie。 Gunjack增加DLC。,"['优点', '：', '整', '体', '做', '的', '非常', '好', '，', '沉浸', '感', '比', '较强', '，', '价格', '合理', '。', '不', '晕', '。', '缺点', '：', '整', '体', '游戏', '时', '间', '有', '点', '短', '，', '并且', '不能', '调整', '难度', '。', '期望', '：', 'steam', '赶紧销', '售', 'EVE', 'Valkyrie', '。', 'Gunjack', '增加', 'DLC', '。']"


In [71]:
def preprocessing4(corpus, stop_word):
    result = []
    for item in tqdm(corpus.iloc[:,10]):
        result.append(movestopwords(item))
    corpus['content_seg_no_stw']=result
    
    return corpus

In [72]:
data_n = preprocessing4(data, stw)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
data.head()

,game_id,useful_num,funny_num,user_name,games_owned,reviews_written,recommended,hours_played,review_date,text,text_seg_monpa_no_stw,content_seg_no_stw
0,280720.0,8,0,76561198206307425,128,51,1,40.3,"15 August, 2019",雖說是搶先體驗，但是玩了這麼久，沒感覺有遇到bug...算是很有誠意了。 漢化上面大部分都好了，漏了一個星球的主線沒有而已，這個是即時制、可調時間速度與暫停的遊戲，也可以很快地讀取、存檔、關閉，因為外星人侵略性不是很大，也沒有軍事部隊，主要是透過製作的道具或是購買股權的方式去吞併削弱對手。 當然也可以選擇故意破壞環境，讓大災變替你把所有對手趕跑之後自己再慢慢重新收拾XDD 可能就背景音樂的部分比較不足而已，其他畫面啦、遊戲內容之類的，因為沒在打廣告，所以只要三百多而已，很值啦。,"['雖說', '是', '搶先', '體驗', '，', '但是', '玩', '了', '這麼', '久', '，', '沒', '感覺', '有', '遇到', 'bug', '...', '算是', '很', '有', '誠意', '了', '。', '漢化', '上面', '大部分', '都', '好了', '，', '漏', '了', '一', '個', '星球', '的', '主線', '沒有', '而已', '，', '這', '個', '是', '即時制', '、', '可', '調', '時間', '速度', '與', '暫停', '的', '遊戲', '，', '也', '可以', '很', '快', '地', '讀取', '、', '存檔', '、', '關閉', '，', '因為', '外星人', '侵略性', '不', '是', '很', '大', '，', '也', '沒有', '軍事', '部隊', '，', '主要', '是', '透過', '製作', '的', '道具', '或是', '購買', '股權', '的', '方式', '去', '吞併', '削弱', '對手', '。', '當然', '也', '可以', '選擇', '故意', '破壞', '環境', '，', '讓', '大', '災變', '替', '你', '把', '所有', '對手', '趕跑', '之後', '自己', '再', '慢慢', '重新', '收拾', 'XDD', '可能', '就', '背景', '音樂', '的', '部分', '比較', '不足', '而已', '，', '其他', '畫面', '啦', '、', '遊戲', '內容', '之類', '的', '，', '因為', '沒', '在', '打廣告', '，', '所以', '只要', '三百多', '而已', '，', '很', '值', '啦', '。']","[', 雖, ', ,, , ', ', ,, , ', 搶, 先, ', ,, , ', 體, 驗, ', ,, , ', ', ,, , ', ', ,, , ', 玩, ', ,, , ', ', ,, , ', 麼, ', ,, , ', 久, ', ,, , ', ', ,, , ', ', ,, , ', 感, 覺, ', ,, , ', ', ,, , ', 遇, ', ,, , ', ', ,, , ', ', ,, , ', 算, ', ,, , ', ', ,, , ', ', ,, , ', 誠, 意, ', ,, , ', ', ...]"
1,410570.0,0,0,76561198424252800,38,2,1,0.7,"7 October, 2019",good,['good'],"[', ']"
2,410570.0,0,0,76561198300261802,16,3,1,0.7,"23 June, 2017",遊戲是挺耐玩的，不過玩得有點累，美術方面，可能因為從手機版移植來，所以遠景有點假。但整體而言，印象還不錯，科幻迷可以考慮。,"['遊戲', '是', '挺', '耐玩', '的', '，', '不過', '玩', '得', '有點', '累', '，', '美術', '方面', '，', '可能', '因為', '從', '手機版', '移植', '來', '，', '所以', '遠景', '有點', '假', '。', '但', '整體', '而言', '，', '印象', '還', '不錯', '，', '科幻迷', '可以', '考慮', '。']","[', 遊, 戲, ', ,, , ', ', ,, , ', 挺, ', ,, , ', 耐, 玩, ', ,, , ', ', ,, , ', ', ,, , ', 過, ', ,, , ', 玩, ', ,, , ', ', ,, , ', 點, ', ,, , ', 累, ', ,, , ', ', ,, , ', 美, 術, ', ,, , ', 方, 面, ', ,, , ', ', ,, , ', ', ,, , ', 因, ', ,, , ', ', ,, , ', 手, 機, 版, ', ,, , ', 移, 植, ', ,, , ', ...]"
3,410570.0,0,0,76561198307052400,80,6,1,6.6,"30 September, 2016",這是我玩過幾款vr遊戲中我個人覺得很不錯的一款遊戲,"['這', '是', '我', '玩', '過', '幾', '款', 'vr', '遊戲', '中', '我', '個人', '覺得', '很', '不錯', '的', '一', '款', '遊戲']","[', ', ,, , ', ', ,, , ', ', ,, , ', 玩, ', ,, , ', 過, ', ,, , ', 幾, ', ,, , ', 款, ', ,, , ', ', ,, , ', 遊, 戲, ', ,, , ', ', ,, , ', ', ,, , ', ', ,, , ', 覺, ', ,, , ', ', ,, , ', 錯, ', ,, , ', ', ,, , ', ', ,, , ', 款, ', ,, , ', 遊, 戲, ']"
4,410570.0,0,0,76561198307075359,102,4,1,9.3,"21 September, 2016",优点：整体做的非常好，沉浸感比较强，价格合理。不晕。缺点：整体游戏时间有点短，并且不能调整难度。期望：steam赶紧销售EVE:Valkyrie。 Gunjack增加DLC。,"['优点', '：', '整', '体', '做', '的', '非常', '好', '，', '沉浸', '感', '比', '较强', '，', '价格', '合理', '。', '不', '晕', '。', '缺点', '：', '整', '体', '游戏', '时', '间', '有', '点', '短', '，', '并且', '不能', '调整', '难度', '。', '期望', '：', 'steam', '赶紧销', '售', 'EVE', 'Valkyrie', '。', 'Gunjack', '增加', 'DLC', '。']","[', 优, 点, ', ,, , ', ', ,, , ', 整, ', ,, , ', 体, ', ,, , ', ', ,, , ', ', ,, , ', 非, 常, ', ,, , ', ', ,, , ', ', ,, , ', 沉, 浸, ', ,, , ', 感, ', ,, , ', ', ,, , ', 较, 强, ', ,, , ', ', ,, , ', 价, 格, ', ,, , ', 合, 理, ', ,, , ', ', ,, , ', ', ,, , ', 晕, ', ,, , ', ', ,, , ', 缺, 点, ', ...]"
